ANALISIS RAPIDO

In [2]:
import sys
from pathlib import Path
import pandas as pd

# Agregamos la carpeta src al path para poder importar
path_inicio = Path.cwd().parent/"src"
sys.path.append(str(path_inicio))
from utilidades.constantes import DATA_PATH, DATA_PROCESADA_PATH, PATRON_MOSTRADOR_PATH, MOSTRADOR_SALIDA_PATH, PATRON_FACTURACION_PATH, FACTURACION_SALIDA_PATH, ARCHIVO_UNIFICADO_PATH


In [3]:
df_unificado = pd.read_csv(ARCHIVO_UNIFICADO_PATH, encoding='utf-8-sig')
df_mostrador = pd.read_csv(MOSTRADOR_SALIDA_PATH, encoding='utf-8-sig')
df_facturacion = pd.read_csv(FACTURACION_SALIDA_PATH, encoding='utf-8-sig')